<a href="https://colab.research.google.com/github/swagathmullangi/learn_agentic_ai/blob/main/crew_ai_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install langchain_community langchain_openai faiss-cpu langchain_tavily langchain sentence-transformers langchain-huggingface langchain-groq crewai crewai_tools

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = "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"

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.ibm.com/think/topics/ai-agents")
web_content = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(web_content)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-mpnet-base-v2")

from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from langchain.agents import Tool
from langchain.tools import tool

@tool
def get_contenxt_from_vector_db(question):
  "get the context about ai agents from the vector db"
  docs = retriever.invoke(question)
  return docs

import random
def get_random_number(input=''):
  "generate random number"
  return random.randint(0, 10)

random_tool = Tool(name="get_random_number", func=get_random_number, description="generate random number")

from langchain.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])


In [ ]:
tools = [search_tool, random_tool, get_contenxt_from_vector_db]

In [ ]:
from langchain_groq import ChatGroq

groq_api = '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'
llm = ChatGroq(groq_api_key=groq_api,
               model = 'llama-3.1-8b-instant')

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory(memory_key="chat_history",
                                  k=3,
                                  return_messages=True)


from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         verbose=True,
                         memory=memory,
                         max_iterations = 2,
                         early_stopping_method="generate")

<ipython-input-7-a00272edbd53>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",
<ipython-input-7-a00272edbd53>:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools,


In [ ]:
agent.invoke("What is current temperature in Bangalore?")



> Entering new AgentExecutor chain...
Question: What is current temperature in Bangalore?
Thought: To get the current temperature in Bangalore, I need to search for accurate and up-to-date information.
Action: tavily_search_results_json
Action Input: "current temperature in Bangalore"
Observation: [{'title': 'Bangalore Weather Today and AQI Level', 'url': 'https://www.businesstoday.in/weather/bangalore-weather-forecast-today', 'content': 'Today’s minimum temperature in Bangalore is recorded at 23°c (degrees celsius), and the maximum temperature is expected to go as high as 34°c (degrees celsius). Throughout the day, the temperature is likely to hover around 31°c (degrees celsius). The wind will blow at the speed of 6.34, and with the gust of 8.34, it will blow at 138 degrees.The sun will rise at around 05:56 AM in the morning, and around 06:38 PM, it will set in the evening. As per the weekly weather forecast of Bangalore, the [...] temperature expected on the different days of the w

{'input': 'What is current temperature in Bangalore?',
 'chat_history': [HumanMessage(content='What is current temperature in Bangalore?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in Bangalore is around 24°C.', additional_kwargs={}, response_metadata={})],
 'output': 'The current temperature in Bangalore is around 24°C.'}

In [ ]:
agent.invoke("What is my current time?")



> Entering new AgentExecutor chain...
Question: What is my current time?
Thought: I can use the system time, but since I don't have direct access to it, I'll use a search engine to find the current time.
Action: tavily_search_results_json
Action Input: current time
Observation: [{'title': 'World Clock — current time around the world', 'url': 'https://24timezones.com/', 'content': 'Consequently, a location observing UTC+3 during standard time is at UTC+4 during DST; a location at\n                    UTC−6 during standard time is at UTC−5 during DST. Clock shifts are usually scheduled on weekends\n                    to lessen disruption of weekday schedules.\n\nHow Daylight Saving Time works\n\nAM/PM [...] The implementation of Greenwich Mean Time was the first step to determine the time zone of other\n                    countries in regard to GMT+0, while the concept of Coordinated Universal Time (UTC) was designed to\n                    provide a more accurate timekeeping system.

{'input': 'What is my current time?',
 'chat_history': [HumanMessage(content='What is current temperature in Bangalore?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in Bangalore is around 24°C.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my current time?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current time in Washington D.C. on May 12, 2025, is 7:24:45 P.M. Eastern Daylight Time (EDT), which is UTC-4.', additional_kwargs={}, response_metadata={})],
 'output': 'The current time in Washington D.C. on May 12, 2025, is 7:24:45 P.M. Eastern Daylight Time (EDT), which is UTC-4.'}

In [ ]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
get_random_number(input='') - generate random number
get_contenxt_from_vector_db(question) - get the context about ai agents from the vector db

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tavily_search_results_json, get_random_number, get_contenxt_from_vector_db]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from crewai.tools import BaseTool
from crewai_tools import PDFSearchTool
from crewai import Task
from crewai import Agent
from crewai import Crew
from crewai import LLM

### Creating Tool

In [ ]:
from pydantic import BaseModel, Field

pdf_file = "/content/1706.03762v7.pdf"

os.environ['GROQ_API_KEY'] = groq_api

# llm = ChatGroq(
#     model = 'llama-3.1-8b-instant'
#     )

llm = LLM(
    model = "groq/llama-3.3-70b-versatile", api_key="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"
)

config = dict(
    embedder = dict(
        provider = "huggingface",
        config = dict(
            model = "sentence-transformers/all-mpnet-base-v2"
        )
    ),
    llm=dict(
          provider="groq",
          config=dict(
              model="llama-3.1-8b-instant",
          ),
    ),
)

class ToolInput(BaseModel):
  question: str = Field(description="question about the document")

# Creating WebSearch tool
class SearchTool(BaseTool):
  name: str = "Search Tool"
  description: str = "Useful for websearch. Use this tool to find the current information required for query"
  args_schema: type = ToolInput

  def _run(self, question: str):
    "use this tool to find the current information required for query"
    search_tool = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])
    return search_tool.run(question)

# Creating Generation tool
class GenerationTool(BaseTool):
  name: str = "Generation Tool"
  description: str = "Useful for generic query. Use this tool to find information based on your own knowledge"
  args_schema: type = ToolInput

  def _run(self, question: str):
    "use this tool to find information based on your own knowledge"
    return llm.invoke(question)

generation_tool = GenerationTool()
web_search_tool = SearchTool()
pdf_search_tool = PDFSearchTool(
    config = config,
    pdf = pdf_file,
)

# pdf_search_tool.run('What the formula for attention score?')

### Creating an Agent

In [ ]:
# Creating Router Agent
router_agent = Agent(
    role = "router",
    goal = "route user question to pdf_search tool, web_search_tool or generation_tool",
    backstory= (
        "you are an expert in rounting user question to tool"
        "use the vectorstore for question regarding transformer "
        "use the web_search for question regarding current events"
        "use the generation for generic query"

    ),
    verbose=True,
    llm = llm,
    allow_delegation=False
)

# Creating Retriever Agent
retriever_agent = Agent(
    role = "retriever",
    goal = "use the information retrieved from the tool to answer the question",
    backstory = (
      "you are an assistant in question answering task"
      "use the retrieved information to answer user's question"
    ),
    verbose=True,
    llm = llm,
    allow_delegation=False
)

### Creating a Task

In [ ]:
router_task = Task(
    description = (
        "analyse the keywords present in question {question}"
        "based on keywords decide whether it is eligible for vectorstore , web-search or generation"
        "return a single word 'vectorstore' if it is eligible for vectorstore"
        "return a single word 'web-search' if it is eligible for web search"
        "return a single word 'generation' if it is eligible for generation"
    ),
    expected_output= (
        "based on question return a single word from ['vectorstore', 'web-search', 'generation']"
    ),
    agent = router_agent,
)

retriever_task = Task(
    description= (
        "based on the response from router task extract information for user question {question} with the help respective tool"
        "if router task output is 'web-search' then call the web_search_tool tool to reterived information"
        "if router task output is 'vectorstore' then call the pdf_search_tool tool to reterived information"
        "if router task output is 'generation' then call the generation_tool tool to reterived information"
    ),
    expected_output=(
        "always analyse the output from 'router task'"
        "return a clear and concise test as reponse after using correct tool "
    ),
    agent = retriever_agent,
    context = [router_task],
    tools = [pdf_search_tool,web_search_tool,generation_tool]
)

### Creating a Crew

In [ ]:
crew_obj = Crew(
    agents = [router_agent,retriever_agent],
    tasks = [router_task,retriever_task],
    verbose = True
)

In [ ]:
input_question = input("Enter the question?: ")
# crew_obj.kickoff(inputs={"question":"what is formula for attention score?"})
crew_obj.kickoff(inputs={"question":input_question})

what is formula for attention score?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4b5125b6-24e6-4de6-b958-61a0bc1955e5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 77ef7a3e-1eb2-40a8-baac-38c1b1366012
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 77ef7a3e-1eb2-40a8-baac-38c1b1366012
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: analyse the keywords present in question what is formula for attention score?based on keywords decide whether it is eligible for vectorstore , web-search or generationreturn a single word 'vectorstore' if it is eligible for vectorstorereturn a single word 'web-search' if it is eligible for web searchreturn a single word 'generation' if it is eligible for generation


# Agent: router
## Final Answer: 
vectorstore




...

🚀 Crew: crew
├── 📋 Task: 77ef7a3e-1eb2-40a8-baac-38c1b1366012
│      Assigned to: router
│      Status: ✅ Completed
│   └── 🤖 Agent: router
│           Status: ✅ Completed
└── 📋 Task: a6ad67ad-58f2-482d-a2be-7801f8f85c96
       Status: Executing Task...
    └── 🤖 Agent: retriever
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 77ef7a3e-1eb2-40a8-baac-38c1b1366012
│      Assigned to: router
│      Status: ✅ Completed
│   └── 🤖 Agent: router
│           Status: ✅ Completed
└── 📋 Task: a6ad67ad-58f2-482d-a2be-7801f8f85c96
       Assigned to: retriever
       Status: ✅ Completed
    └── 🤖 Agent: retriever
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a6ad67ad-58f2-482d-a2be-7801f8f85c96                                                                     │
│  Agent: retriever                                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4b5125b6-24e6-4de6-b958-61a0bc1955e5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='The formula for attention score is: Attention(Q, K, V) = softmax(QKT √dk )V (1) where Q, K, and V are matrices representing the queries, keys, and values, respectively, and dk is the dimension of the keys. The attention function computes the dot products of the query with all keys, divides each by √dk, and applies a softmax function to obtain the weights on the values.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="analyse the keywords present in question what is formula for attention score?based on keywords decide whether it is eligible for vectorstore , web-search or generationreturn a single word 'vectorstore' if it is eligible for vectorstorereturn a single word 'web-search' if it is eligible for web searchreturn a single word 'generation' if it is eligible for generation", name=None, expected_output="based on question return a single word from ['vectorstore', 'web-search', 'generation']", summary='analyse the keywords present in question wha